In [ ]:
# import everything and define a test runner function
from importlib import reload
from math import ceil, log

import block, helper, network

from block import (
    Block,
    GENESIS_BLOCK_HASH,
    TESTNET_GENESIS_BLOCK_HASH,
)
from helper import (
    hash256,
    merkle_parent,
    merkle_parent_level,
    merkle_root,
    read_varint,
    run,
)
from merkleblock import MerkleTree
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    SimpleNode,
    VersionMessage,
    BLOCK_DATA_TYPE,
)
from tx import Tx

### Exercise 1

#### Check the cheat sheet for the network message structure.

#### 1.1. Parse this message
```
f9beb4d976657261636b000000000000000000005df6e0e2
```

#### 1.2. Make [these tests](/edit/session7/network.py) pass.
```
network.py:NetworkTest:test_parse
network.py:NetworkTest:test_serialize
```

In [ ]:
# Exercise 1.1

msg = bytes.fromhex('f9beb4d976657261636b000000000000000000005df6e0e2')

# first 4 are network magic
# next 12 are command
# next 4 are payload length
# next 4 are checksum
# rest is payload

In [ ]:
# Exercise 1.2

reload(network)
run(network.NetworkEnvelopeTest('test_parse'))
run(network.NetworkEnvelopeTest('test_serialize'))

### Exercise 2

#### 2.1. Make [this test](/edit/session7/network.py) pass.
```
network.py:VersionMessageTest:test_serialize
```

In [ ]:
# Exercise 2.1

reload(network)
run(network.VersionMessageTest('test_serialize'))

### Exercise 3

#### 3.1. Make [this test](/edit/session7/network.py) pass.
```
network.py:GetHeadersMessageTest:test_serialize
```

#### 3.2. Make [this test](/edit/session7/network.py) pass.
```
network.py:HeadersMessageTest:test_parse
```

In [ ]:
# Exercise 3.1

reload(network)
run(network.GetHeadersMessageTest('test_serialize'))

In [ ]:
# Exercise 3.2

reload(network)
run(network.HeadersMessageTest('test_parse'))

In [ ]:
# Handshake Example

from network import SimpleNode, VersionMessage

node = SimpleNode('tbtc.programmingblockchain.com', testnet=True, logging=True)

version = VersionMessage()
node.send(version.command, version.serialize())
print(node.wait_for_commands([b'verack']))

### Exercise 4

#### 4.1. Make [this test](/edit/session7/network.py) pass.
```
network.py:SimpleNodeTest:test_handshake
```

In [ ]:
# Exercise 4.1

reload(network)
run(network.SimpleNodeTest('test_handshake'))

In [ ]:
# Block Header Download Example

from network import GetHeadersMessage, HeadersMessage, SimpleNode

node = SimpleNode('btc.programmingblockchain.com', testnet=False)
node.handshake()
last_block_hash = GENESIS_BLOCK_HASH
count = 1
for _ in range(20):
    getheaders = GetHeadersMessage(start_block=last_block_hash)
    node.send(getheaders.command, getheaders.serialize())
    headers_envelope = node.wait_for_commands([b'headers'])
    headers_message = HeadersMessage.parse(headers_envelope.stream())
    for b in headers_message.blocks:
        if not b.check_pow():
            raise RuntimeError('bad proof of work at block {}'.format(count))
        if last_block_hash != GENESIS_BLOCK_HASH and b.prev_block != last_block_hash:
            raise RuntimeError('discontinuous block at {}'.format(count))
        count += 1
        last_block_hash = b.hash()
        if count % 2016 == 0:
            print(b.id())

### Exercise 5

#### 5.1. Download the first 40,000 blocks for testnet and validate them.


In [ ]:
# Exercise 5.1

# connect to tbtc.programmingblockchain.com
# handshake
# set the last block hash to the TESTNET_GENESIS_BLOCK_HASH
# set the block count to 1
# loop until we we get 40,000 blocks
    # create a GetHeadersMessage starting from the last block we have
    # send the getheaders command
    # wait for the headers command in response
    # parse the network envelope that comes back
    # loop through the blocks from the headers message
        # check the proof of work
        # the prev_block of the current block should be the last block
        # increment the block count
        # set the last block hash
        # print the block header every 2016 blocks (difficulty adjustment)

In [ ]:
# Merkle Parent Example

tx_hash0 = bytes.fromhex('c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5')
tx_hash1 = bytes.fromhex('c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5')

parent = hash256(tx_hash0+tx_hash1)
print(parent.hex())

### Exercise 6

#### 6.1. Calculate the Merkle parent of these hashes:
```
f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0
3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181
```

#### 6.2. Make [this test](/edit/session7/helper.py) pass
```
helper.py:HelperTest:test_merkle_parent
```

In [ ]:
# Exercise 6.1

hex_hash1 = 'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0'
hex_hash2 = '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181'

# bytes.fromhex to get the bin hashes
# hash256 the combination
# hex() to see the result

In [ ]:
# Exercise 6.2

reload(helper)
run(helper.HelperTest('test_merkle_parent'))

In [ ]:
# Merkle Parent Level Example

from helper import merkle_parent

hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
]
hashes = [bytes.fromhex(x) for x in hex_hashes]
if len(hashes) % 2 == 1:
    hashes.append(hashes[-1])
parent_level = []
for i in range(0, len(hex_hashes), 2):
    parent = merkle_parent(hashes[i], hashes[i+1])
    print(parent.hex())
    parent_level.append(parent)

### Exercise 7

#### 7.1. Calculate the next Merkle Parent Level given these hashes
```
8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd
7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800
ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7
68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069
43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27
```

#### 7.2. Make [this test](/edit/session7/helper.py) pass.
```
helper.py:HelperTest:test_merkle_parent_level
```

In [ ]:
# Exercise 7.1

hex_hashes = [
    '8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd',
    '7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800',
    'ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7',
    '68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069',
    '43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27',
]

# bytes.fromhex to get all the hashes in binary
# if the number of hashes is odd, duplicate the last one
# initialize parent level
# skip by two: use range(0, len(hashes), 2)
    # calculate merkle_parent of i and i+1 hashes
    # print the hash's hex
    # add parent to parent level

In [ ]:
# Exercise 7.2

reload(helper)
run(helper.HelperTest('test_merkle_parent_level'))

In [ ]:
# Merkle Root Example

from helper import merkle_parent_level

hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
    '7d37b3d54fa6a64869084bfd2e831309118b9e833610e6228adacdbd1b4ba161',
    '8118a77e542892fe15ae3fc771a4abfd2f5d5d5997544c3487ac36b5c85170fc',
    'dff6879848c2c9b62fe652720b8df5272093acfaa45a43cdb3696fe2466a3877',
    'b825c0745f46ac58f7d3759e6dc535a1fec7820377f24d4c2c6ad2cc55c0cb59',
    '95513952a04bd8992721e9b7e2937f1c04ba31e0469fbe615a78197f68f52b7c',
    '2e6d722e5e4dbdf2447ddecc9f7dabb8e299bae921c99ad5b0184cd9eb8e5908',
    'b13a750047bc0bdceb2473e5fe488c2596d7a7124b4e716fdd29b046ef99bbf0',
]

current_level = [bytes.fromhex(x) for x in hex_hashes]
while len(current_level) > 1:
    current_level = merkle_parent_level(current_level)
print(current_level[0].hex())

### Exercise 8

#### 8.1. Calculate the Merkle Root given these hashes
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 8.2. Make [this test](/edit/session7/helper.py) pass.
```
helper.py:HelperTest:test_merkle_root
```

In [ ]:
# Exercise 8.1

hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]

# bytes.fromhex to get all the hashes in binary
# initialize current level to be the hashes
# loop until current_level has only 1 element
    # make the current level the parent level
# print the root's hex

In [ ]:
# Exercise 8.2

reload(helper)
run(helper.HelperTest('test_merkle_root'))

In [ ]:
# Block Merkle Root Example

from helper import merkle_root

tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
]
current_level = [bytes.fromhex(x)[::-1] for x in tx_hex_hashes]
print(merkle_root(current_level)[::-1].hex())

### Exercise 9

#### 9.1. Validate the merkle root for this block on Testnet:
Block Hash:
```
0000000000000451fa80fcdb243b84c35eaae215a85a8faa880559e8239e6f20
```

Transaction Hashes:
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 9.2. Make [this test](/edit/session7/block.py) pass.
```
block.py:BlockTest:test_validate_merkle_root
```

In [ ]:
# Exercise 9.1

want = '4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301'

tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]

# bytes.fromhex and reverse ([::-1]) to get all the hashes in binary
# get the merkle root
# see if the reversed root is the same as the wanted root

In [ ]:
# Exercise 9.2

reload(block)
run(block.BlockTest('test_validate_merkle_root'))

### Exercise 10

#### 10.1. Validate the merkle root for this block on Testnet via network protocol:
Block Hash:
```
0000000000044b01a9440b34f582fe171c7b8642fedd0ebfccf8fdf6a1810900
```

In [ ]:
# Exercise 10.1

block_hex = '0000000000044b01a9440b34f582fe171c7b8642fedd0ebfccf8fdf6a1810900'
block_hash = bytes.fromhex(block_hex)

# connect to tbtc.programmingblockchain.com on testnet
# handshake

# create a GetDataMessage
# add_data on the message (BLOCK_DATA_TYPE, block_hash)
# send the getdata command

# wait for the block command in response
# get the stream from the envelope
# block message is block header first, so parse a Block
# check the proof of work
# next the block message has the number of transactions so read_varint that
# initialize the tx_hashes list
# loop through that number of times
    # parse the transaction
    # add the transaction hash to the tx_hashes list

# set the block's tx_hashes to the ones we've gathered
# validate the tx_hashes
# print the merkle root hex

In [ ]:
# Merkle Tree Example

total = 16

max_depth = ceil(log(total, 2))
merkle_tree = []
for depth in range(max_depth + 1):
    num_items = ceil(total / 2**(max_depth - depth))
    level_hashes = [None] * num_items
    merkle_tree.append(level_hashes)

for level in merkle_tree:
    print(level)

In [ ]:
# Merkle Tree Populating and Navigating Example

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
    "f9dbfafc3af3400954975da24eb325e326960a25b87fffe23eef3e7ed2fb610e",
]
tree = MerkleTree(len(hex_hashes))
tree.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
tree.nodes[3] = merkle_parent_level(tree.nodes[4])
tree.nodes[2] = merkle_parent_level(tree.nodes[3])
tree.nodes[1] = merkle_parent_level(tree.nodes[2])
tree.nodes[0] = merkle_parent_level(tree.nodes[1])
print(tree)

In [ ]:
# Merkle Tree Populating Example #2

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
    "f9dbfafc3af3400954975da24eb325e326960a25b87fffe23eef3e7ed2fb610e",
]
tree = MerkleTree(len(hex_hashes))
tree.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
while tree.root() is None:
    if tree.is_leaf():
        tree.up()
    else:
        left_hash = tree.get_left_node()
        right_hash = tree.get_right_node()
        if left_hash is None:
            tree.left()
        elif right_hash is None:
            tree.right()
        else:
            tree.set_current_node(merkle_parent(left_hash, right_hash))
            tree.up()
print(tree)

In [ ]:
# Merkle Tree Populating Example #3

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
]
tree = MerkleTree(len(hex_hashes))
tree.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
while tree.root() is None:
    if tree.is_leaf():
        tree.up()
    else:
        left_hash = tree.get_left_node()
        if left_hash is None:
            tree.left()
        elif tree.right_exists():
            right_hash = tree.get_right_node()
            if right_hash is None:
                tree.right()
            else:
                tree.set_current_node(merkle_parent(left_hash, right_hash))
                tree.up()
        else:
            tree.set_current_node(merkle_parent(left_hash, left_hash))
            tree.up()
print(tree)